## 背景
&#8195;&#8195;GB/T34090.2-2017《互动广告第2部分:投放验证要求》是由中国广告协会组织行业各方力量共同制定的国家标准。此份标准规定了互联网广告投放监测的一系列重要规则，包括：广告监测指标项及其计算要求（广告曝光量、广告独立访问者数、广告点击量、广告独立点击者数、触达次数、互联网毛点评、点击率）和广告验证（广告无效流量、广告可见性、广告品牌安全）相关标准内容。第三方监测企业在此标准的指导下，协助广告主和数字媒体使用统一的衡量标准，提升数字广告运营效率。

&#8195;&#8195;在上述标准中，无效流量被分为一般无效流量（GIVT）和复杂无效流量（SIVT）。GIVT是具有明显非真人特征，或者触碰了行业上下游形成共识的可疑行为的流量，如：机器人爬虫产生的流量、自动重复刷新网页、UA参数缺失等。这恰恰也是“无效流量不能等同于作弊”的原因。无效流量标准不仅严厉打击恶意作弊行为，也通过对流量特征的一系列规定，促进行业规范、有序运转。SIVT是具有高伪装能力，需要通过大数据、人工智能、统计学等分析手段才能识别的恶意可疑流量，如：同一个设备短时间内跨越众多城市，机器人操控设备自动浏览或翻页。

&#8195;&#8195;针对广协说明，三方监测秒针制定了相关sivt的规则用于识别和过滤复杂无效流量，对此fancy也进行了相关工作探究秒针所给出规则的详细信息，并使用于日常流量反作弊规则中。具体规则见[秒针反作弊规则整合](https://km.fancydsp.com/pages/viewpage.action?pageId=34865320)，具体报表见[秒针SIVT监控报表](http://easyreport.fancydsp.com/projects/1/dashboards/268/preview/)

In [7]:
# 准备工作
import numpy  as np
import pandas as pd
#连接presto库
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

# 本地则使用代理，默认socks5端口号为18888
import os
if os.environ.get('JUPYTERENV') != 'release':
    import socks
    import socket
    socks.set_default_proxy(socks.SOCKS5, "localhost", 18888)
    socket.socket = socks.socksocket

presto_engine = create_engine('presto://192.168.28.111:10080/hive/dwd')

In [14]:
sivt_df = pd.read_sql('''
with data_detail as (
    select a.*,  b.order_name
    , if(action='imp', 1, 0 ) as imp
    , if(action='clk', 1, 0 ) as clk
    from 
        (select thisdate
        , cast(replace(timestamp, '_', ' ') as timestamp) as timestp
        , request_id as r_id
        , vendor_id as vid
        , uaid as ua_id
        , mz_ip as ip
        , cusid, spid, os
        , upper(trim(case
            when length(m5)  = 36 and device_type <> 5 then m5
            when length(m2)  = 32 and device_type <> 5 then m2
            when length(m11) >= 15 and device_type <> 5 then m11
            when length(m1a) = 32 then m1a
            when length(m6a) = 32 then m6a
            when length(m6)  = 32 then m6
            else cast(random(100000000000000) as varchar)
            end)) as did
        , upper(trim(if(m1a = '__ANDROIDID__', '', m1a))) as aid
        , upper(trim(if(m2  = '__IMEI__'     , '', m2)))  as imei
        , upper(trim(if(m5  = '__IDFA__'     , '', m5)))  as idfa
        , upper(trim(if(length(m6a) > 0 and m6a <> '__MAC__', m6a, ''))) as mac
        , upper(trim(if(length(m6)  > 0 and m6 <> '__MAC1__' and vendor_id <> 30, m6, '')))  as mac1
        , upper(trim(if(m11 = '__OAID__', '', m11))) as oaid
        , md as model
        , substr(replace(timestamp, '_', ' '), 1, 16) as minute_str
        , device_type
        , case when device_type=5 then 'OTT' when  (device_type=4 or device_type is null) then 'PC' else 'MOB' end as iterm
        , advertiser_id
        , t2.order_id
        , case when action in('imp','incrs_imp','render_imp') then 'imp' else 'clk' end as action
        , row_number() over(partition by request_id, action, spid) as num 
        from rpt.base_customer_monitor_data t1
        left join
        (select o.order_id, adg.id
            FROM u6rds.amo."order" o left join u6rds.ad_fancy.campaign_amo_order_mapping o_map
            on o.order_id=o_map.order_id left join u6rds.ad_fancy.ad_group adg
            on o_map.campaign_id=adg.campaign_id
            WHERE o.start_date <= CAST('2021-03-22' AS timestamp)
            AND o.end_date >= CAST('2021-03-22' AS timestamp)
            AND o.order_id >= 1000
            AND o.order_id <> 1024
            AND o.source=1
            AND o.status <> -1) t2
        on t1.adgroup_id = t2.id
        where thisdate='2021-03-22'
        and action in('imp','incrs_imp','clk','render_imp')) a
        join
        (SELECT o.order_id
        , o.order_name
        , o.advertiser_id
        --, regexp_extract(ome.imp_url, '(?<=/k=).*?(?=&|$)')
        --, regexp_extract(ome.imp_url, '(?<=p=).*?(?=&|$)')
        , array_join(array_distinct(split(array_join(array_agg(regexp_extract(ome.imp_url, '(?<=/k=).*?(?=&|$)')), ','),',')),',') as cusid
        , array_join(array_distinct(split(array_join(array_agg(regexp_extract(ome.imp_url, '(?<=p=).*?(?=&|$)')), ','),',')),',') as spid
        FROM u6rds.amo."order" o
            LEFT JOIN u6rds.ad_fancy.advertiser adv ON o.advertiser_id = adv.id
            LEFT JOIN u6rds.amo.order_monitoring om ON o.order_id = om.order_id
            LEFT JOIN u6rds.amo.order_monitoring_ext ome ON om.id = ome.monitoring_id
        WHERE o.start_date <= CAST('2021-03-22' AS timestamp)
            AND o.end_date >= CAST('2021-03-22' AS timestamp)
            AND o.order_id >= 1000
            AND o.order_id <> 1024
            AND o.source=1
            AND o.status <> -1
            AND (o.main_nmps = '秒针' OR other_nmps = '秒针' OR ome.imp_url like '%%miaozhen.com%%')
            and regexp_extract(ome.imp_url, '(?<=p=).*?(?=&|$)') is not null
        group by 1, 2, 3) b
        on a.advertiser_id=b.advertiser_id and a.order_id=b.order_id and contains(split(b.spid,','), a.spid)
    where a.num=1)

                       
select thisdate, 'dsp' as type, vid, iterm, advertiser_id, order_id, order_name, sum(imp) as imp, sum(clk) as clk
    , sum(case
        when d_is_did_lost    = 1  then imp
        when d_is_did_illegal = 1  then imp
        when did_minute_imp > 5 or aid_minute_imp > 5 or imei_minute_imp > 5 or oaid_minute_imp > 5 or mac_minute_imp > 5 or mac1_minute_imp > 5 or idfa_minute_imp > 5  then imp
        when diff_imp_scds     < 16 then imp
        when d_did_md_cnt > 2 or d_aid_md_cnt > 2 or d_imei_md_cnt > 2 or d_oaid_md_cnt > 2 or d_mac_md_cnt > 2 or d_mac1_md_cnt > 2 or d_idfa_md_cnt > 2  then imp
        when d_did_os_cnt > 1 or d_aid_os_cnt > 1 or d_imei_os_cnt > 1 or d_oaid_os_cnt > 1 or d_mac_os_cnt > 1 or d_mac1_os_cnt > 1 or d_idfa_os_cnt > 1  then imp
        when d_imei_aid_cnt > 2 or a_imei_aid_cnt > 2 or d_mac_aid_cnt > 2 or d_mac1_aid_cnt > 2 or a_mac_aid_cnt > 2 or a_mac1_aid_cnt > 2 then imp
        when d_aid_imei_cnt > 2 or i_aid_imei_cnt > 2 or d_mac_imei_cnt > 2 or d_mac1_imei_cnt > 2 or i_mac_imei_cnt > 2 or i_mac1_imei_cnt > 2 then imp
        when d_is_did_illegal_2os = 1 then imp
        when d_idfa_mac_cnt > 2 or d_idfa_mac1_cnt > 2 or m_idfa_mac_cnt > 2 or m_idfa_mac1_cnt > 2 or d_imei_mac_cnt > 2 or d_imei_mac1_cnt > 2 or m_imei_mac_cnt > 2 or m_imei_mac1_cnt > 2 then imp
        when d_mac_idfa_cnt > 2 or d_mac1_idfa_cnt > 2 or i_mac_idfa_cnt > 2 or i_mac1_idfa_cnt > 2 then imp
        else 0
      end) as sivt_imp
    , sum(case
        when d_is_did_lost    = 1  then clk
        when d_is_did_illegal = 1  then clk
        when did_minute_clk > 4 or aid_minute_clk > 4 or imei_minute_clk > 4 or oaid_minute_clk > 4 or mac_minute_clk > 4 or mac1_minute_clk > 4 or idfa_minute_clk > 4  then clk
--         when diff_clk_scds     < 16 then clk
        when d_did_md_cnt > 2 or d_aid_md_cnt > 2 or d_imei_md_cnt > 2 or d_oaid_md_cnt > 2 or d_mac_md_cnt > 2 or d_mac1_md_cnt > 2 or d_idfa_md_cnt > 2  then clk
        when d_did_os_cnt > 1 or d_aid_os_cnt > 1 or d_imei_os_cnt > 1 or d_oaid_os_cnt > 1 or d_mac_os_cnt > 1 or d_mac1_os_cnt > 2 or d_idfa_os_cnt > 1   then clk
        when d_imei_aid_cnt > 2 or a_imei_aid_cnt > 2 or d_mac_aid_cnt > 2 or d_mac1_aid_cnt > 2 or a_mac_aid_cnt > 2 or a_mac1_aid_cnt > 2 then clk
        when d_aid_imei_cnt > 2 or i_aid_imei_cnt > 2 or d_mac_imei_cnt > 2 or d_mac1_imei_cnt > 2 or i_mac_imei_cnt > 2 or i_mac1_imei_cnt > 2 then clk
        when d_is_did_illegal_2os = 1 then clk
        when d_idfa_mac_cnt > 2 or d_idfa_mac1_cnt > 2 or m_idfa_mac_cnt > 2 or m_idfa_mac1_cnt > 2 or d_imei_mac_cnt > 2 or d_imei_mac1_cnt > 2 or m_imei_mac_cnt > 2 or m_imei_mac1_cnt > 2 then clk
        when d_mac_idfa_cnt > 2 or d_mac1_idfa_cnt > 2 or i_mac_idfa_cnt > 2 or i_mac1_idfa_cnt > 2 then clk
        else 0
      end) as sivt_clk
    , sum(if(d_is_did_lost = 1, imp, 0))                                                                                                                     as mb_without_device_id_imp  
    -- 设备ID缺失
    , sum(if(d_is_did_illegal = 1, imp, 0))                                                                                                                  as deviceId_illegal_imp  
    -- 设备ID回参不规范
    , sum(if(did_minute_imp > 5 or aid_minute_imp > 5 or imei_minute_imp > 5 or oaid_minute_imp > 5 or mac_minute_imp > 5 or mac1_minute_imp > 5 or idfa_minute_imp > 5, imp, 0))   as excessive_imp  
    --  曝光过高
    , sum(if(diff_imp_scds < 16, imp, 0))                                                                                                                    as continuous_imp  
    -- 曝光碰撞
    , sum(if(d_did_md_cnt > 2 or d_aid_md_cnt > 2 or d_imei_md_cnt > 2 or d_oaid_md_cnt > 2 or d_mac_md_cnt > 2 or d_mac1_md_cnt > 2 or d_idfa_md_cnt > 2, imp, 0))               as multi_device_imp  
    -- 设备ID不匹配
    , sum(if(d_did_os_cnt > 1 or d_aid_os_cnt > 1 or d_imei_os_cnt > 1 or d_oaid_os_cnt > 1 or d_mac_os_cnt > 1 or d_mac1_os_cnt > 2 or d_idfa_os_cnt > 1, imp, 0))               as multi_os_imp  
    -- 操作系统不匹配
    , sum(if(d_imei_aid_cnt > 2 or a_imei_aid_cnt > 2 or d_mac_aid_cnt > 2 or d_mac1_aid_cnt > 2 or a_mac_aid_cnt > 2 or a_mac1_aid_cnt > 2, imp, 0))                                                    as multi_android_id_imp 
    -- AndroidID不匹配
    , sum(if(d_aid_imei_cnt > 2 or i_aid_imei_cnt > 2 or d_mac_imei_cnt > 2 or d_mac1_imei_cnt > 2 or i_mac_imei_cnt > 2 or i_mac1_imei_cnt > 2, imp, 0))                                                  as multi_imei_imp 
    -- IMEI不匹配
    , sum(if(d_is_did_illegal_2os = 1, imp, 0))                                                                                                              as illeagal_os_imp 
    -- 操作系统非法
    , sum(if(d_idfa_mac_cnt > 2 or d_idfa_mac1_cnt > 2 or m_idfa_mac_cnt > 2 or m_idfa_mac1_cnt > 2 or d_imei_mac_cnt > 2 or d_imei_mac1_cnt > 2 or m_imei_mac_cnt > 2 or m_imei_mac1_cnt > 2, imp, 0))       as multi_mac_imp  
    -- MAC不匹配
    , sum(if(d_mac_idfa_cnt > 2 or d_mac1_idfa_cnt > 2 or i_mac_idfa_cnt > 2 or i_mac1_idfa_cnt > 2, imp, 0))                                                                                              as multi_idfa_imp 
    -- IDFA不匹配

    -- SIVT Click Detail
    , sum(if(d_is_did_lost = 1, clk, 0))                                                                                                                     as mb_without_device_id_clk  
    -- 点击设备缺失
    , sum(if(d_is_did_illegal = 1, clk, 0))                                                                                                                  as deviceId_illegal_clk  
    -- 点击设备回参不规范
    , sum(if(did_minute_clk > 4 or aid_minute_clk > 4 or imei_minute_clk > 4 or oaid_minute_clk > 4 or mac_minute_clk > 4 or mac1_minute_clk > 4 or idfa_minute_clk > 4, clk, 0))   as excessive_clk 
    -- 点击频繁
    , sum(if(d_did_md_cnt > 2 or d_aid_md_cnt > 2 or d_imei_md_cnt > 2 or d_oaid_md_cnt > 2 or d_mac_md_cnt > 2 or d_mac1_md_cnt > 2 or d_idfa_md_cnt > 2, clk, 0))               as multi_device_clk  
    -- 点击设备ID不匹配
    , sum(if(d_did_os_cnt > 1 or d_aid_os_cnt > 1 or d_imei_os_cnt > 1 or d_oaid_os_cnt > 1 or d_mac_os_cnt > 1 or d_mac1_os_cnt > 1 or d_idfa_os_cnt > 1, clk, 0))               as multi_os_clk 
    -- 点击操作系统不匹配
    , sum(if(d_imei_aid_cnt > 2 or a_imei_aid_cnt > 2 or d_mac_aid_cnt > 2 or d_mac1_aid_cnt > 2 or a_mac_aid_cnt > 2 or a_mac1_aid_cnt > 2, clk, 0))                                                    as multi_android_id_clk 
    -- 点击AndroidID不匹配
    , sum(if(d_aid_imei_cnt > 2 or i_aid_imei_cnt > 2 or d_mac_imei_cnt > 2 or d_mac1_imei_cnt > 2 or i_mac_imei_cnt > 2 or i_mac1_imei_cnt > 2, clk, 0))                                                  as multi_imei_clk 
    -- 点击IMEI不匹配
    , sum(if(d_is_did_illegal_2os = 1, clk, 0))                                                                                                              as illeagal_os_clk  
    -- 点击操作系统非法
    , sum(if(d_idfa_mac_cnt > 2 or d_idfa_mac1_cnt > 2 or m_idfa_mac_cnt > 2 or m_idfa_mac1_cnt > 2 or d_imei_mac_cnt > 2 or d_imei_mac1_cnt > 2 or m_imei_mac_cnt > 2 or m_imei_mac1_cnt > 2, clk, 0))              as multi_mac_clk  
    -- 点击MAC不匹配
    , sum(if(d_mac_idfa_cnt > 2 or d_mac1_idfa_cnt > 2 or i_mac_idfa_cnt > 2 or i_mac1_idfa_cnt > 2, clk, 0))                                                                                              as multi_idfa_clk 
    -- 点击DIFA不匹配


from
(
    select *
        -- Device ID Lost
        , max(is_did_lost)  over (partition by did, thisdate) as d_is_did_lost
        -- Illegal/Nonstandard DeviceID
        , max(is_did_illegal) over (partition by did, thisdate) as d_is_did_illegal

        -- multi device type
        , max(did_md_cnt)   over (partition by did, thisdate) as d_did_md_cnt
        , max(aid_md_cnt)   over (partition by did, thisdate) as d_aid_md_cnt
        , max(imei_md_cnt)  over (partition by did, thisdate) as d_imei_md_cnt
        , max(oaid_md_cnt)  over (partition by did, thisdate) as d_oaid_md_cnt
        , max(mac_md_cnt)   over (partition by did, thisdate) as d_mac_md_cnt
        , max(mac1_md_cnt)  over (partition by did, thisdate) as d_mac1_md_cnt
        , max(idfa_md_cnt)  over (partition by did, thisdate) as d_idfa_md_cnt

        -- multi os
        , max(did_os_cnt)   over (partition by did, thisdate) as d_did_os_cnt
        , max(aid_os_cnt)   over (partition by did, thisdate) as d_aid_os_cnt
        , max(imei_os_cnt)  over (partition by did, thisdate) as d_imei_os_cnt
        , max(oaid_os_cnt)  over (partition by did, thisdate) as d_oaid_os_cnt
        , max(mac_os_cnt)   over (partition by did, thisdate) as d_mac_os_cnt
        , max(mac1_os_cnt)  over (partition by did, thisdate) as d_mac1_os_cnt
        , max(idfa_os_cnt)  over (partition by did, thisdate) as d_idfa_os_cnt

        -- multi android id
        , max(imei_aid_cnt) over (partition by did, thisdate) as d_imei_aid_cnt
        , max(imei_aid_cnt) over (partition by w_aid, thisdate) as a_imei_aid_cnt
        , max(mac_aid_cnt)  over (partition by did, thisdate) as d_mac_aid_cnt
        , max(mac_aid_cnt)  over (partition by w_aid, thisdate) as a_mac_aid_cnt
        , max(mac1_aid_cnt) over (partition by did, thisdate) as d_mac1_aid_cnt
        , max(mac1_aid_cnt) over (partition by w_aid, thisdate) as a_mac1_aid_cnt

        -- multi imei
        , max(aid_imei_cnt) over (partition by did, thisdate) as d_aid_imei_cnt
        , max(aid_imei_cnt) over (partition by w_imei, thisdate) as i_aid_imei_cnt
        , max(mac_imei_cnt) over (partition by did, thisdate) as d_mac_imei_cnt
        , max(mac_imei_cnt) over (partition by w_imei, thisdate) as i_mac_imei_cnt
        , max(mac1_imei_cnt) over (partition by did, thisdate) as d_mac1_imei_cnt
        , max(mac1_imei_cnt) over (partition by w_imei, thisdate) as i_mac1_imei_cnt

        -- Illegal ID To OS
        , max(is_did_illegal_2os) over (partition by did, thisdate) as d_is_did_illegal_2os

        -- multi mac
        , max(idfa_mac_cnt) over (partition by did, thisdate) as d_idfa_mac_cnt
        , max(idfa_mac_cnt) over (partition by w_mac, thisdate) as m_idfa_mac_cnt
        , max(idfa_mac1_cnt) over (partition by did, thisdate) as d_idfa_mac1_cnt
        , max(idfa_mac1_cnt) over (partition by w_mac1, thisdate) as m_idfa_mac1_cnt
        , max(imei_mac_cnt) over (partition by did, thisdate) as d_imei_mac_cnt
        , max(imei_mac_cnt) over (partition by w_mac, thisdate) as m_imei_mac_cnt
        , max(imei_mac1_cnt) over (partition by did, thisdate) as d_imei_mac1_cnt
        , max(imei_mac1_cnt) over (partition by w_mac1, thisdate) as m_imei_mac1_cnt

        --  multi idfa
        , max(mac_idfa_cnt) over (partition by did, thisdate) as d_mac_idfa_cnt
        , max(mac_idfa_cnt) over (partition by w_idfa, thisdate) as i_mac_idfa_cnt
        , max(mac1_idfa_cnt) over (partition by did, thisdate) as d_mac1_idfa_cnt
        , max(mac1_idfa_cnt) over (partition by w_idfa, thisdate) as i_mac1_idfa_cnt

        -- 曝光过高
        , sum(imp) over (partition by did, minute_str)    as did_minute_imp
        , sum(imp) over (partition by w_aid, minute_str)  as aid_minute_imp
        , sum(imp) over (partition by w_imei, minute_str) as imei_minute_imp
        , sum(imp) over (partition by w_oaid, minute_str) as oaid_minute_imp
        , sum(imp) over (partition by w_mac, minute_str)  as mac_minute_imp
        , sum(imp) over (partition by w_mac1, minute_str) as mac1_minute_imp
        , sum(imp) over (partition by w_idfa, minute_str) as idfa_minute_imp

        -- 点击频繁
        , sum(clk) over (partition by did, minute_str)    as did_minute_clk
        , sum(clk) over (partition by w_aid, minute_str)  as aid_minute_clk
        , sum(clk) over (partition by w_imei, minute_str) as imei_minute_clk
        , sum(clk) over (partition by w_oaid, minute_str) as oaid_minute_clk
        , sum(clk) over (partition by w_mac, minute_str)  as mac_minute_clk
        , sum(clk) over (partition by w_mac1, minute_str) as mac1_minute_clk
        , sum(clk) over (partition by w_idfa, minute_str) as idfa_minute_clk

        -- 曝光碰撞
        , if(pre_imp_diff_scds < post_imp_diff_scds, pre_imp_diff_scds, post_imp_diff_scds) as diff_imp_scds
        , if(pre_clk_diff_scds < post_clk_diff_scds, pre_clk_diff_scds, post_clk_diff_scds) as diff_clk_scds
    from
    (
        select *
            -- multi device type
            , approx_distinct(ua_id)  over (partition by did, thisdate)    as did_md_cnt
            , approx_distinct(ua_id)  over (partition by w_aid, thisdate)  as aid_md_cnt
            , approx_distinct(ua_id)  over (partition by w_imei, thisdate) as imei_md_cnt
            , approx_distinct(ua_id)  over (partition by w_oaid, thisdate) as oaid_md_cnt
            , approx_distinct(ua_id)  over (partition by w_mac, thisdate)  as mac_md_cnt
            , approx_distinct(ua_id)  over (partition by w_mac1, thisdate) as mac1_md_cnt
            , approx_distinct(ua_id)  over (partition by w_idfa, thisdate) as idfa_md_cnt

            -- multi os
            , approx_distinct(os)     over (partition by did, thisdate)    as did_os_cnt
            , approx_distinct(os)     over (partition by w_aid, thisdate)  as aid_os_cnt
            , approx_distinct(os)     over (partition by w_imei, thisdate) as imei_os_cnt
            , approx_distinct(os)     over (partition by w_oaid, thisdate) as oaid_os_cnt
            , approx_distinct(os)     over (partition by w_mac, thisdate)  as mac_os_cnt
            , approx_distinct(os)     over (partition by w_mac1, thisdate) as mac1_os_cnt
            , approx_distinct(os)     over (partition by w_idfa, thisdate) as idfa_os_cnt

            -- multi android id
            , approx_distinct(m_aid)  over (partition by w_imei, thisdate) as imei_aid_cnt -- multi android id (imei)
            , approx_distinct(m_aid)  over (partition by w_mac, thisdate)  as mac_aid_cnt  -- multi android id (mac)
            , approx_distinct(m_aid)  over (partition by w_mac1, thisdate) as mac1_aid_cnt -- multi android id (mac1)

            -- multi imei
            , approx_distinct(m_imei) over (partition by w_aid, thisdate)  as aid_imei_cnt -- multi imei (android id)
            , approx_distinct(m_imei) over (partition by w_mac, thisdate)  as mac_imei_cnt -- multi imei (mac)
            , approx_distinct(m_imei) over (partition by w_mac1, thisdate) as mac1_imei_cnt -- multi imei (mac1)

            -- multi mac
            , approx_distinct(m_mac)  over (partition by w_idfa, thisdate) as idfa_mac_cnt -- multi mac (idfa)
            , approx_distinct(m_mac)  over (partition by w_imei, thisdate) as imei_mac_cnt -- multi mac (imei)
            , approx_distinct(m_mac1) over (partition by w_idfa, thisdate) as idfa_mac1_cnt -- multi mac1 (idfa)
            , approx_distinct(m_mac1) over (partition by w_imei, thisdate) as imei_mac1_cnt -- multi mac1 (imei)

            -- multi idfa
            , approx_distinct(m_idfa) over (partition by w_mac, thisdate)  as mac_idfa_cnt -- multi idfa (mac)
            , approx_distinct(m_idfa) over (partition by w_mac1, thisdate) as mac1_idfa_cnt -- multi idfa (mac1)

            -- continuous imp/clk
            , date_diff('second', pre_imp_timestp, timestp)  as pre_imp_diff_scds
            , date_diff('second', timestp, post_imp_timestp) as post_imp_diff_scds
            , date_diff('second', pre_clk_timestp, timestp)  as pre_clk_diff_scds
            , date_diff('second', timestp, post_clk_timestp) as post_clk_diff_scds
        from
        (
                select *
                , if(iterm <> 'PC' and length(did)<15, 1, 0) as is_did_lost
                , case
                    when length(aid) = 32 and regexp_like(aid, '^[0-9A-Fa-f]+$') then 0
                    when length(aid) > 0  then 1
                    when length(imei) = 32 and regexp_like(imei, '^[0-9A-Fa-f]+$') then 0
                    when length(imei) > 0  then 1
                    when length(idfa) = 36 and regexp_like(idfa, '^[0-9A-Fa-f-]+$') then 0
                    when length(idfa) > 0  then 1
                    when length(mac)  = 32 and regexp_like(mac, '^[0-9A-Fa-f]+$') then 0
                    when length(mac)  > 0  then 1
                    when length(mac1) = 32 and regexp_like(mac1, '^[0-9A-Fa-f]+$') then 0
                    when length(mac1) > 0 then 1
                    when length(oaid) in (16, 32, 36, 64) and regexp_like(oaid, '^[0-9A-Fa-f-]+$') then 0
                    when length(oaid) > 0  then 1
                    else 0
                  end as is_did_illegal
                ,  case
                    when os = 0 and idfa <> '' then 1
                    when os = 1 and aid  <> '' then 1
                    when os = 1 and imei <> '' then 1
                    when os = 1 and oaid <> '' then 1
                    else 0
                  end as is_did_illegal_2os
                , case
                    when mac = 'E3F5536A141811DB40EFD6400F1D0A4E' then NULL
                    when mac = '35B9AB5A36F3234DD26DB357FD4A0DC1' then NULL
--                     when mac = '528C8E6CD4A3C6598999A0E9DF15AD32' then NULL
--                     when mac = '0F607264FC6318A92B9E13C65DB7CD3C' then NULL
                    when mac = 'D41D8CD98F00B204E9800998ECF8427E' then NULL
                    when mac = '' then NULL
                    else mac
                end as m_mac
                , case
                    when mac = 'E3F5536A141811DB40EFD6400F1D0A4E' then did
                    when mac = '35B9AB5A36F3234DD26DB357FD4A0DC1' then did
--                     when mac = '528C8E6CD4A3C6598999A0E9DF15AD32' then did
--                     when mac = '0F607264FC6318A92B9E13C65DB7CD3C' then did
                    when mac = 'D41D8CD98F00B204E9800998ECF8427E' then did
                    when mac = '' then did
                    else mac
                end as w_mac
                , case
--                     when mac1 = 'E3F5536A141811DB40EFD6400F1D0A4E' then NULL
--                     when mac1 = '35B9AB5A36F3234DD26DB357FD4A0DC1' then NULL
                    when mac1 = '528C8E6CD4A3C6598999A0E9DF15AD32' then NULL
                    when mac1 = '0F607264FC6318A92B9E13C65DB7CD3C' then NULL
                    when mac1 = 'D41D8CD98F00B204E9800998ECF8427E' then NULL
                    when mac1 = '' then NULL
                    else mac1
                end as m_mac1
                , case
--                     when mac1 = 'E3F5536A141811DB40EFD6400F1D0A4E' then did
--                     when mac1 = '35B9AB5A36F3234DD26DB357FD4A0DC1' then did
                    when mac1 = '528C8E6CD4A3C6598999A0E9DF15AD32' then did
                    when mac1 = '0F607264FC6318A92B9E13C65DB7CD3C' then did
                    when mac1 = 'D41D8CD98F00B204E9800998ECF8427E' then did
                    when mac1 = '' then did
                    else mac1
                end as w_mac1
                , if(aid <> '' , aid, NULL) as m_aid   , if(aid <> '', aid, did)   as w_aid
                , if(imei <> '' and device_type <> 5, imei, NULL) as m_imei, if(imei <> '' and device_type <> 5, imei, did) as w_imei
                , if(idfa <> '' and device_type <> 5, idfa, NULL) as m_idfa, if(idfa <> '' and device_type <> 5, idfa, did) as w_idfa
                , if(oaid <> '' , oaid, NULL) as m_oaid, if(oaid <> '', oaid, did) as w_oaid
                , LAG(timestp, 1,  cast('1970-01-01 00:00:00' as timestamp)) over (partition by did, action, thisdate order by timestp) as pre_imp_timestp
                , LEAD(timestp, 1, cast('2099-12-31 00:00:00' as timestamp)) over (partition by did, action, thisdate order by timestp) as post_imp_timestp
                , LAG(timestp, 1,  cast('1970-01-01 00:00:00' as timestamp)) over (partition by did, action, thisdate order by timestp) as pre_clk_timestp
                , LEAD(timestp, 1, cast('2099-12-31 00:00:00' as timestamp)) over (partition by did, action, thisdate order by timestp) as post_clk_timestp
            from data_detail a
        ) b
    ) c
)d
group by 1, 2, 3, 4, 5, 6, 7
''', presto_engine)
sivt_df

,thisdate,type,vid,iterm,advertiser_id,order_id,order_name,imp,clk,sivt_imp,...,mb_without_device_id_clk,deviceId_illegal_clk,excessive_clk,multi_device_clk,multi_os_clk,multi_android_id_clk,multi_imei_clk,illeagal_os_clk,multi_mac_clk,multi_idfa_clk
0,2021-03-22,dsp,118,MOB,20168,3145,韩剧TV-浩御-阿迪达斯-3.5-4.10,243094,12094,6693,...,297,35,0,0,0,0,0,0,0,0
1,2021-03-22,dsp,30,MOB,2003397,3113,DSP-IPG-埃森哲FY21-2021.3.1-2021.03.31,92349,684,36,...,0,0,0,0,0,0,0,0,0,0
2,2021-03-22,dsp,52,MOB,2003397,3113,DSP-IPG-埃森哲FY21-2021.3.1-2021.03.31,2797,60,8,...,0,0,0,0,0,0,0,0,0,0
3,2021-03-22,dsp,118,MOB,2003506,3094,韩剧TV-AOD-美赞臣铂睿-2.22-4.4,18958,1278,757,...,39,3,0,0,0,0,0,0,0,0
4,2021-03-22,dsp,29,PC,2003397,3113,DSP-IPG-埃森哲FY21-2021.3.1-2021.03.31,69804,156,0,...,0,0,0,0,0,0,0,0,0,0
5,2021-03-22,dsp,10118,MOB,20168,3145,韩剧TV-浩御-阿迪达斯-3.5-4.10,49274,2934,8,...,0,0,0,0,0,0,0,0,0,0
6,2021-03-22,dsp,157,MOB,2003397,3113,DSP-IPG-埃森哲FY21-2021.3.1-2021.03.31,1001,47,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-03-22,dsp,157,MOB,2003506,3094,韩剧TV-AOD-美赞臣铂睿-2.22-4.4,63994,4144,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-03-22,dsp,81,MOB,2003397,3113,DSP-IPG-埃森哲FY21-2021.3.1-2021.03.31,13929,491,124,...,0,0,0,0,0,0,0,0,0,0
9,2021-03-22,dsp,118,MOB,20168,3095,韩剧TV-浩御-阿迪达斯-3.1-3.28,46,2,3,...,0,0,0,0,0,0,0,0,0,0
